![image](../images/kdd24-logo-small.jpeg)

# Hands-on Tutorial
## Domain-Driven LLM Development: Insights into RAG and Fine-Tuning Practices
### Lab 1.1: Naive RAG
#### Summary: 
Objective:
This notebook demonstrates the implementation of a Naive Retrieval-Augmented Generation (RAG) pipeline using the following components:

- Embedding Model: `amazon.titan-embed-text-v2:0`

- Vector Database: `Epsilla's vector database`

- Large Language Model (LLM): `meta.llama3-8b-instruct-v1:0`

The data we use is BONTONSTORESINC_04_20_2018-EX-99.3-AGENCY AGREEMENT and ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT from the Contract Understanding Atticus Dataset (https://github.com/TheAtticusProject/cuad)

We also demostrate evaluating RAG correctness with 3 metrics:

- cosine similarity
- token_overlap_recall
- rouge_l_recall

---

#### Load the PDF file as text string

In [22]:
!pip install PyMuPDF

3197.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [23]:
import fitz

In [25]:
# Function to extract text using PyMuPDF
def extract_text_from_pdf_mupdf(pdf_path):
    text = ""
    document = fitz.open(pdf_path)
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

In [26]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/BONTONSTORESINC_04_20_2018-EX-99.3-AGENCY AGREEMENT.PDF")
pdf_text_mupdf[:2000]  # Displaying the first 2000 characters to get an overview of the content

'EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court 

#### Chunk the document with uniform length strings

In [27]:
def chunk_text(text, chunk_size, overlap):
    """
    Chunk text into smaller segments with a specified chunk size and overlap.

    Parameters:
    - text (str): The text to be chunked.
    - chunk_size (int): The size of each chunk.
    - overlap (int): The number of characters that overlap between chunks.

    Returns:
    - List[str]: A list of text chunks.
    """
    if chunk_size <= overlap:
        raise ValueError("Chunk size must be greater than overlap")

    chunks = []
    start = 0
    end = chunk_size

    while start < len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
        end = start + chunk_size

    return chunks

In [28]:
chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

210


['EXHIBIT 99.3\nCase 18-10248-\nMFW\nDoc 632-1\nFiled 04/18/18\nPage 2 of 60\nAGENCY AGREEMENT\nThis Agency Agreement (“Agreement”) is made as of April 18, 2018, by and between The Bon-Ton Stores, Inc. and its associated chapter 11 debtors in\npossession (collectively, “Merchant”),1 on the one hand, and (a) a contractual joint venture comprised of GA Retail, Inc. (“GA”) and Tiger Capital Group, LLC\n(“Tiger” and collectively with GA, the “Agent”) and (b) Wilmington Savings Fund Society, FSB, as the indenture agent and collateral trustee for the 8.00%\nsecond-lien senior secured notes due 2021 (the “Second-Lien Notes”) issued by BTDS, on the other hand (in such capacities, the “Notes Trustee” and\ncollectively with Agent, “Purchaser”). Purchaser and Merchant are collectively the “Parties.”\nSection 1. Recitals\nWHEREAS, on February 4, 2018, the entities comprising Merchant commenced ten voluntary chapter 11 bankruptcy cases (the “Bankruptcy Cases”) in\nthe United States Bankruptcy Court

#### Setup the embedding service

Utilize the amazon.titan-embed-text-v2:0 model to convert the documents into dense vector embeddings. Each document will be transformed into a high-dimensional vector representation that captures its semantic content.

In [29]:
import boto3
import json

In [30]:
# Initialize the Bedrock client
client = boto3.client('bedrock-runtime', region_name='us-west-2')

In [1]:
# Function to embed text
def embed_text(input_text):
    # Create the request payload
    payload = {
        "inputText": input_text,
        "dimensions": 512,  # Specify the desired dimension size
        "normalize": True  # Whether to normalize the output embeddings
    }

    # Invoke the model
    response = client.invoke_model(
        body=json.dumps(payload),
        modelId='amazon.titan-embed-text-v2:0',  # Specify the Titan embedding model
        accept='application/json',
        contentType='application/json'
    )

    # Get the embedding result
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')
    return embedding

# Print the embedding
print(embed_text(chunks[0]))

NameError: name 'chunks' is not defined

#### Setup vector database, and build knowledge base from the PDF document

Store the generated embeddings in Epsilla's vector database. The database will be configured to allow efficient similarity search, which is critical for the retrieval step in RAG.

Ensure that each embedding is stored with a reference to its original document so that it can be retrieved later.

In [10]:
!pip install pyepsilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 51.9 MB/s eta 0:00:00


In [14]:
!sh ../setup.sh

Using default tag: latest
latest: Pulling from epsilla/vectordb

29bcf202: Pulling fs layer 
555358b1: Pulling fs layer 
b700ef54: Pulling fs layer 
a17be611: Pulling fs layer 
934d8b21: Pulling fs layer 
5cb974f3: Pulling fs layer 
6f850f31: Pulling fs layer 
42c631d3: Pulling fs layer 
09595de6: Pull complete 421MB/421MBkBBAExtracting  304.7kB/304.7kBDigest: sha256:fee11976ed6428a5b4c62e95c9ddebd5083abc17e28e8d34fde625f7de35ebfb
Status: Downloaded newer image for epsilla/vectordb:latest
docker.io/epsilla/vectordb:latest
0d6b12fa7c4ba3b23d370e6274bf900b8e869e6e314dc5d88f096023e424781a
Check Vector DB:
curl: (52) Empty reply from server



In [15]:
from pyepsilla import vectordb
## connect to vectordb
db = vectordb.Client(
  host='localhost',
  port='8888'
)

[INFO] Connected to localhost:8888 successfully.


In [18]:
db.unload_db("kdd_lab1_rag")
db.load_db(db_name="kdd_lab1_rag", db_path="/tmp/kdd_lab1_rag")

(200, {'statusCode': 200, 'message': 'Load/Create kdd_lab1_rag successfully.'})

In [19]:
db.use_db(db_name="kdd_lab1_rag")
db.create_table(
  table_name="NaiveRAG",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

(200, {'statusCode': 200, 'message': 'Create NaiveRAG successfully.'})

In [ ]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

In [ ]:
db.insert("NaiveRAG", records)

#### Setup interence service

Use `meta.llama3-8b-instruct-v1:0` as LLM for completion.

In [17]:
def generate(prompt):
    # Create the request payload
    payload = {
        "prompt": prompt,
        "temperature": 0,  # Adjust the randomness of the output
        "max_gen_len": 128
    }

    # Initialize the Bedrock runtime client
    client = boto3.client('bedrock-runtime', region_name='us-west-2')

    # Invoke the model
    response = client.invoke_model(
        modelId='meta.llama3-8b-instruct-v1:0',
        contentType='application/json',
        accept='application/json',
        body=json.dumps(payload)
    )
    
    byte_response = response['body'].read()
    json_string = byte_response.decode('utf-8')

    # Get the chat response
    response_body = json.loads(json_string)
    chat_response = response_body.get('generation')

    return chat_response

# Example usage
input_text = "How are you?"
prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{input_text}[/INST]
"""

print(generate(prompt))

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.

#### Setup the RAG pipeline

When a query is received, embed the query using the same amazon.titan-embed-text-v2:0 model to generate a query vector.

Perform a similarity search in Epsilla's vector database to retrieve the most relevant documents based on the cosine similarity between the query vector and the stored document embeddings.

Pass the retrieved documents (or their summaries) along with the original query to meta.llama3-8b-instruct-v1:0.

The LLM will generate a response that is contextually informed by the retrieved documents, effectively augmenting the generation with relevant information.

In [ ]:
def basic_retriever(table_name, question, top_k):
    code, resp = db.query(
        table_name=table_name,
        query_field="Embedding",
        query_vector=embed_text(question),
        limit=top_k
    )
    return resp["result"]
basic_retriever("NaiveRAG", "What's the agreement date?", 5)

In [ ]:
import re
def naive_rag(table_name, question):
    docs = basic_retriever(table_name, question, 5)
    docs_str = "------------------------\n"
    for doc in docs:
        docs_str += doc["Doc"] + "\n------------------------\n"
    prompt = f"""
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

Your answer should be grounded by the information provided in the documents below.
Don't make up answers.
Don't explain your thought process.
Directly answer the question in concise way.

<documents>
{docs_str}
</documents>
<</SYS>>

{question}[/INST]
"""
    cleaned_response = re.sub(r'</?[^>]+>', '', generate(prompt))
    return prompt, cleaned_response

data_augmented_prompt, answer = naive_rag("NaiveRAG", "What's the agreement date?")

print(answer)

#### Evaluation on question answer accuracy

In [ ]:
# Extract the text from the PDF using PyMuPDF
pdf_text_mupdf = extract_text_from_pdf_mupdf("../lab-data/ENERGOUSCORP_03_16_2017-EX-10.24-STRATEGIC ALLIANCE AGREEMENT.PDF").replace('\xa0', ' ')
pdf_text_mupdf  # Displaying the first 2000 characters to get an overview of the content

In [ ]:
chunks = chunk_text(pdf_text_mupdf, 1024, 256)
print(len(chunks))
chunks[:2]

In [ ]:
db.create_table(
  table_name="NaiveRAG_Evaluation",
  table_fields=[
    {"name": "ID", "dataType": "INT", "primaryKey": True},
    {"name": "Doc", "dataType": "STRING"},
    {"name": "Embedding", "dataType": "VECTOR_FLOAT", "dimensions": 512}
  ]
)

In [ ]:
records = [
    {
        "ID": index,
        "Doc": text,
        "Embedding": embed_text(text)
    }
    for index, text in enumerate(chunks)
]
records[:2]

In [ ]:
db.insert("NaiveRAG_Evaluation", records)

In [ ]:
basic_retriever("NaiveRAG_Evaluation", "What's the agreement date?", 5)

In [ ]:
naive_rag("NaiveRAG_Evaluation", "According to the contract, how is the renewal term handled after the initial term expires, including details on automatic extensions or extensions requiring prior notice?	")

In [ ]:
import pandas as pd

# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for question in questions:
    data_augmented_prompt, generated_answer = naive_rag("NaiveRAG_Evaluation", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
!pip install continuous-eval

In [ ]:
!pip install tiktoken sentence_transformers

In [ ]:
from continuous_eval.metrics.generation.text import DeterministicAnswerCorrectness
from sentence_transformers import SentenceTransformer, util

evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
import csv

# Specify the file name
csv_file = '../lab-data/naive_rag_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file

In [ ]:
# Load the CSV file
file_path = '../lab-data/ENERGOUSCORP_qa_test.csv'
df = pd.read_csv(file_path)

# Assuming the CSV has 'Question' and 'Answer' columns
questions = df['question'].tolist()
answers = df['answer'].tolist()

questions, answers

In [ ]:
data_augmented_prompts = []
generated_answers = []
for question in questions:
    data_augmented_prompt, generated_answer = naive_rag("NaiveRAG_Evaluation", question)
    data_augmented_prompts.append(data_augmented_prompt)
    generated_answers.append(generated_answer)

generated_answers

In [ ]:
evaluation_results = []
for i in range(len(questions)):
    ground_truth_embedding = embed_text(answers[i])
    answer_embedding = embed_text(generated_answers[i])
    cosine_similarity = util.cos_sim(ground_truth_embedding, answer_embedding)[0][0].item()
    
    datum = {
        "answer": generated_answers[i],
        "ground_truth_answers": [
            answers[i]
        ]
    }
    metric = DeterministicAnswerCorrectness()
    eval_result = metric(**datum)
    evaluation_results.append({
        "question": data_augmented_prompts[i],
        "ref_answer": answers[i],
        "response": generated_answers[i],
        "semantic_similarity": cosine_similarity,
        "token_overlap_recall": eval_result["token_overlap_recall"],
        "rouge_l_recall": eval_result["rouge_l_recall"]
    })

evaluation_results

In [ ]:
# Specify the file name
csv_file = '../lab-data/naive_rag_test_result.csv'

# Write the data to a CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=evaluation_results[0].keys())
    writer.writeheader()
    writer.writerows(evaluation_results)

csv_file